In [ ]:
import pandas as pd
import numpy as np
import matplotliine.pyplot as plt
import seaborn as sns

### Import Dataframes

In [ ]:
#redacted

### Construct Summary Dataframe

In [ ]:
#Subset as needed to work with basline isolates and CK618, CK618 + bactrim

#subset LHR data
LHR_LB.Phage.unique()
LHR_LB_Cocks = LHR_LB[(LHR_LB.Phage == "CK618") | (LHR_LB.Phage == "CK618_bactrim")]
LHR_LB_Cocks = LHR_LB_Cocks[LHR_LB_Cocks.Strain != "LB"]
LHR_LB_AUC = LHR_LB_Cocks[["Phage", "Strain", "AUC Ratio"]]
LHR_LB_AUC = LHR_LB_AUC.rename(columns = {"AUC Ratio" : "AUC_Ratio_LB"})

LHR_urine_Cocks = LHR_urine[(LHR_urine.Phage == "CK618") | (LHR_urine.Phage == "CK618_bactrim")]
LHR_urine_Cocks = LHR_urine_Cocks[LHR_urine_Cocks.Strain != "Urine"]
LHR_urine_Cocks = LHR_urine_Cocks[LHR_urine_Cocks["Avg Control Endpoint OD"] > 0.4]
LHR_urine_AUC = LHR_urine_Cocks[["Phage", "Strain", "AUC Ratio"]]
LHR_urine_AUC = LHR_urine_AUC.rename(columns = {"AUC Ratio" : "AUC_Ratio_urine"})

#LHR
LHR_AUC = pd.merge(LHR_LB_AUC, LHR_urine_AUC, how="outer", on=["Strain","Phage"])
LHR_AUC = LHR_AUC[["Strain", "Phage", "AUC_Ratio_LB", "AUC_Ratio_urine"]]

In [ ]:
#Plaquing Data

#will need to consider whether using the maxes is best here
plaquing_just_maxes = plaquing[["Strain", "max_titer_wZOC", "max_EOP"]]
plaquing_just_maxes = plaquing_just_maxes.rename(columns = {"max_EOP" : "max_EOP_noZOC"})

plaquing_all = plaquing[["Strain", "Avg_Titer_wZOC", "max_titer_wZOC", "max_EOP"]]

#convert to log titers
plaquing_all.iloc[:, [1,2]] = plaquing_all.iloc[:, [1,2]].apply(np.log10)
plaquing_all.replace(-np.inf, 0, inplace=True)
plaquing_all.columns=["Strain", "Avg_logTiter_wZOC", "max_logtiter_wZOC", "max_EOP"]

LHR_plaquing = pd.merge(LHR_AUC, plaquing_all, on="Strain", how="left")

In [ ]:
#CFU reduction
CFUred_Cocks = CFUred[(CFUred.Phage == "CK618") | (CFUred.Phage == "CK618_bactrim") | (CFUred.Phage == "No Phage")]

CFUred_grouped = CFUred_Cocks.groupby(["Phage", "Strain", "Timepoint"])["log_titer"].mean().reset_index()

CFUred_grouped.to_csv("CFUred_averages.csv", index=False)

#Pivot that bad boy
CFU_pivoted = CFUred_grouped.pivot(
    columns = "Timepoint",
    index = ["Phage", "Strain"],
    values = "log_titer").reset_index()

CFU_pivoted.columns = ["Phage", "Strain", "logCFU_0h", "logCFU_4h", "logCFU_24h"]

#Take some log differences for the CFU reductions
# this is awful bc of plate kill
#CFU_pivoted["4h_logCFU_reduction"] = CFU_pivoted.logCFU_0h - CFU_pivoted.logCFU_4h
No_Phage = CFU_pivoted[CFU_pivoted.Phage == "No Phage"].reset_index()
No_Phage_twice = pd.concat([No_Phage, No_Phage], ignore_index=True)
# CFU_pivoted["4h_logCFU_reduction"] = No_Phage_twice.logCFU_0h - CFU_pivoted.logCFU_4h
# CFU_pivoted["24h_logCFU_reduction"] = No_Phage_twice.logCFU_0h - CFU_pivoted.logCFU_24h
#CFU_pivoted["0h_NoPhage"] = No_Phage_twice.logCFU_0h
CFU_pivoted["4h_NoPhage_vs_Phage"] = No_Phage_twice.logCFU_4h - CFU_pivoted.logCFU_4h
CFU_pivoted["24h_NoPhage_vs_Phage"] = No_Phage_twice.logCFU_24h - CFU_pivoted.logCFU_24h

in_vitros = pd.merge(LHR_plaquing, CFU_pivoted, on=["Strain", "Phage"], how="left")

In [ ]:
#Merge in in vivo data
in_vivo.rename(columns={"Baseline_Iso" : "Strain"}, inplace=True)
in_vivo["Phage"] = "CK618_bactrim"
Clinical_Summary = pd.merge(in_vitros, in_vivo, on=["Strain", "Phage"], how="left")
#get those patients in there
strain_patient = cnum_clinic[["subject_id", "Name.."]]
strain_patient = strain_patient.rename(columns={"subject_id": "Patient", "Name.." : "Strain"})
strain_patient = strain_patient[["Strain", "Patient"]]
strain_patient.Strain = strain_patient.Strain.str.upper()
Clinical_Summary = pd.merge(strain_patient, Clinical_Summary, on="Strain", how="left")
Clinical_Summary = Clinical_Summary.drop("Patient_y", axis=1)
Clinical_Summary = Clinical_Summary.rename(columns={"Patient_x": "Patient"})
Clinical_Summary = Clinical_Summary.sort_values(by=["Strain", "Phage"], ascending=False)

Clinical_Summary.to_csv("Clinical_Summary.csv", index=False)

#Going to subset to just CK618 + bactrim for my own sanity in fillng the powerpoint out
Clinical_618bactrim = Clinical_Summary[Clinical_Summary.Phage != "CK618"]
Clinical_618bactrim = Clinical_618bactrim.sort_values(by= "Patient")
Clinical_618bactrim.to_csv("Clinical_CK618_bactrim.csv", index=False)

In [ ]:
#just baseline isolates

#drop C000901 aand add C000902
baseline_isos['Strain'] = baseline_isos['Strain'].replace({"C000901": "C000902"})

BaselineIsos_Summary = pd.merge(baseline_isos, Clinical_Summary, how="left", on=["Strain"])
BaselineIsos_Summary = BaselineIsos_Summary.drop("Patient_y", axis=1)
BaselineIsos_Summary = BaselineIsos_Summary.rename(columns={"Patient_x": "Patient"})
BaselineIsos_Summary = BaselineIsos_Summary.sort_values(by=["Strain", "Phage"], ascending=False)
BaselineIsos_Summary.to_csv("BaselineIsos_Summary.csv", index=False)

### Correlative Analysis

In [ ]:
# Calculate correlation matrix
BaselineIsos_noPatient = BaselineIsos_Summary.drop(columns={"Patient"})

#drop the maxCFUdrop anytime column
BaselineIsos_noPatient = BaselineIsos_Summary.drop(columns={"Patient_maxCFUdrop_anyTime"})

correlation_matrix = BaselineIsos_noPatient.corr()

# Display the correlation matrix
print("Correlation Matrix:")
correlation_matrix

# Visualize the correlation matrix
plt.matshow(correlation_matrix)
plt.xticks(range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation="vertical")
plt.yticks(range(len(correlation_matrix.columns)), correlation_matrix.columns)
plt.colorbar()
plt.show()
correlation_matrix.to_csv("VItroVivo_correlation.csv")

In [ ]:
#One-off get the sequencing type 
cnum_sampleid = cnum_clinic[["Name..", "sample_id"]]

abx_suscep = pd.read_csv(#redacted.tsv, sep="\t")
abx_suscep = abx_suscep.rename(columns={"grp_id" : "sample_id"})

cnum_abx = pd.merge(cnum_sampleid, abx_suscep, on = "sample_id", how="left")
cnum_abx.to_csv("cnum_abx.csv", index=False)

#get some counts

cnum_abx_subset = cnum_abx [["Name..", "susceptibility_bactrim"]]
cnum_abx_subset = cnum_abx_subset.rename(columns={"Name.." : "Strain"})
cnum_abx_subset_unique = cnum_abx_subset.drop_duplicates()
#print(cnum_abx_subset_unique["susceptibility_bactrim"].value_counts())

#unique isos
unique_abx = pd.merge(unique_isos, cnum_abx_subset_unique, on="Strain", how="left")
print(unique_abx["susceptibility_bactrim"].value_counts())
print(len(unique_isos))

In [ ]:
#unique isolate summaries
unique_isos_summary = pd.merge(unique_isos, Clinical_618bactrim, on=["Strain", "Patient"], how="left")
unique_isos_summary.to_csv("uniqueisos_618bactrim.csv", index=False)


In [ ]:
# Bar chart using DataFrame
unique_isos_summary = unique_isos_summary.sort_values(by= "Strain", ascending=False)

# Set a larger figure size
plt.figure(figsize=(12,8))

# Conditional color: red for values above the threshold, green otherwise
colors = ['red' if value < 9 else 'gray' for value in unique_isos_summary['Avg_logTiter_wZOC']]

unique_isos_summary.plot(x='Strain', y='Avg_logTiter_wZOC', kind='barh', color=colors, legend=False)

# # Adding labels and title
# plt.xlabel('Categories')
plt.xlabel('log10 PFU/mL')
plt.title('Avg log10 Phage Titer on Ph2 Part 1 Genetically Unique Isolates')

# Adjusting y-axis tick labels fontsize
plt.yticks(fontsize=8)

# Display the plot
plt.show()

In [ ]:
BaselineIsos_618bactrim = BaselineIsos_Summary[BaselineIsos_Summary["Phage"] == "CK618_bactrim"]
BaselineIsos_618bactrim.to_csv("baselineIsos_618bactrim.csv", index=False)
print(len(BaselineIsos_618bactrim.Patient.unique()))

In [ ]:
badat4 = CFUred_bactrim618[CFUred_bactrim618['4h_NoPhage_vs_Phage'] < 3]

In [ ]:
#CFU Reduction Summary Graph

#Just ck618 bactrim comparison
CFUred_bactrim618 = CFU_pivoted[CFU_pivoted["Phage"] == "CK618_bactrim"]

#heat map
columns_to_plot = ['4h_NoPhage_vs_Phage', '24h_NoPhage_vs_Phage']

# Create a horizontally oriented and skinnier heatmap for selected columns
fig, ax = plt.subplots(figsize=(5,8))  # Adjust the width by changing the first value in figsize


# Create a heatmap
ax = sns.heatmap(CFUred_bactrim618[columns_to_plot], annot=True, cmap='viridis', fmt='.2f', linewidths=.5)

# Set the tick labels
ax.set_yticklabels(CFUred_bactrim618.Strain, rotation=0)
ax.set_xticklabels(["4h log red. from cells", "24h log red. from cells"], rotation=0, fontsize=9)

# Set a title for the heatmap
ax.set_title('CFU Reduction on Post-Restart Baseline Isolates')

# Show the plot
plt.show()

#analyze just the 24h point for a bit
cfumL_24h = CFUred_bactrim618[CFUred_bactrim618['logCFU_24h'] < 3.31]

In [ ]:
CFUred_618bactrim = CFUred_grouped[CFUred_grouped["Phage"] == "CK618_bactrim"]

In [ ]:
#Scatter plots

# Extract the columns for the scatter plot
x_data = BaselineIsos_noPatient['4h_NoPhage_vs_Phage']
y_data = BaselineIsos_noPatient['Patient_maxCFUdrop']

# Create a scatter plot
plt.scatter(x_data, y_data, color='blue', marker='o', label='Scatter Plot')

# Set labels and title
plt.xlabel('CFU reduction 4h log difference')
plt.ylabel('Patient maxCFUdrop')
plt.title('4h CFU reduction logDiff vs Patient max CFU drop ')


# Show the plot
plt.show()

### Drop Pre-Pause Isolates for Correlation

In [ ]:
six_prepause_isolates = ['C000675', 'C000678', 'C000681', 'C000686', 'C000687', 'C000692']

BaselineIsos_618bactrim_noprepause = BaselineIsos_618bactrim[~BaselineIsos_618bactrim["Strain"].isin(six_prepause_isolates)]

# Calculate correlation matrix
BaselineIsos_618bactrim_noprepause_noPatient = BaselineIsos_618bactrim_noprepause.drop(columns={"Patient"})
correlation_matrix = BaselineIsos_618bactrim_noprepause_noPatient.corr()

# Display the correlation matrix
print("Correlation Matrix:")
correlation_matrix

# Visualize the correlation matrix
plt.matshow(correlation_matrix)
plt.xticks(range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation="vertical")
plt.yticks(range(len(correlation_matrix.columns)), correlation_matrix.columns)
plt.colorbar()
plt.show()
correlation_matrix

#Doesn't look like subsetting out the pre-pause guys made much of a difference at all. Which makes sense bc they have a lot of missing data

In [ ]:
#Scatter plots


# Extract the columns for the scatter plot
x_data = BaselineIsos_618bactrim_noprepause_noPatient['AUC_Ratio_LB']
y_data = BaselineIsos_618bactrim_noprepause_noPatient['Avg_logTiter_wZOC']

# Create a scatter plot
plt.scatter(x_data, y_data, color='blue', marker='o', label='Scatter Plot')

# Set labels and title
plt.xlabel('AUC Ratio LB')
plt.ylabel('Plaquing log Titer')
plt.title('Plaquing log Titer vs 4h CFU Reduction (r = -0.64)')

# Show the plot
plt.show()

### Getting MiTT Patients

In [ ]:
patients_not_cured = #redacted

#106008 Does not have a day 10 result
MiTT = #redacted
MiTT_minus3nongrowers = #redacted
print("The number of MiTT patients is: " + str(len(MiTT)))
print("The total number of patients is: " + str(len(BaselineIsos_618bactrim.Patient)))
        
really_close_not_MiTT = #redacted

MiTT_patients = BaselineIsos_618bactrim_noprepause[BaselineIsos_618bactrim_noprepause["Patient"].isin(MiTT)]
MiTT_patients.to_csv(#redacted, index=False)
#MiTT_patients_nolowgrow = BaselineIsos_618bactrim_noprepause[BaselineIsos_618bactrim_noprepause["Patient"].isin(MiTT_minus3nongrowers)]

# Calculate correlation matrix
MiTT_patients_noPatient = MiTT_patients.drop(columns={"Patient"})
MiTT_patients_noPatient = MiTT_patients_noPatient.drop(columns={"Patient_maxCFUdrop_anyTime"})
#MiTT_patients_nolowgrow = MiTT_patients_nolowgrow.drop(columns={"Patient"})
correlation_matrix = MiTT_patients_noPatient.corr()

# Display the correlation matrix
print("Correlation Matrix:")
correlation_matrix

# Visualize the correlation matrix
plt.matshow(correlation_matrix)
plt.xticks(range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation="vertical")
plt.yticks(range(len(correlation_matrix.columns)), correlation_matrix.columns)
plt.colorbar(location="right")
plt.show()
correlation_matrix

#Doesn't look like subsetting out the pre-pause patients made much of a difference at all. Which makes sense bc they have a lot of missing data


In [ ]:
all_timepoints_noten = ["Patient_0h_logCFU", "Patient_4h_logCFU", "Patient_24h_logCFU", "Day3_logCFU", "Day4_logCFU", "Day5_logCFU", "Day10_logCFU"]
MiTT_patients["raw_baseline_CFU"] = 10 ** (MiTT_patients["Patient_0h_logCFU"])
MiTT_patients["raw_max_CFU"] = 10 ** MiTT_patients[all_timepoints_noten].max(axis=1)
MiTT_patients["raw_max_drop"] = 10 ** (MiTT_patients["Patient_maxCFUdrop_anyTime"])

MiTT_patients["normalized_drop"] = (MiTT_patients["raw_max_drop"])/MiTT_patients["raw_max_CFU"]
MiTT_patients["log10_normalized_drop"] = (MiTT_patients["Patient_maxCFUdrop_anyTime"]/ (MiTT_patients[all_timepoints_noten].max(axis=1)))

#This analysis is stetchy currently. If I'm going to normalize, it probably needs to be from a consistent reference point for all strains, which would be baseline. Hmm


In [ ]:
#Scatter plots

# Extract the columns for the scatter plot
x_data = MiTT_patients['Avg_logTiter_wZOC']
y_data = MiTT_patients['4h_NoPhage_vs_Phage']

# Create a scatter plot
plt.scatter(x_data, y_data, color='blue', marker='o', label='Scatter Plot')

# Set labels and title
plt.xlabel('Plaquing log Titer')
plt.ylabel('4h CFU Reduction')
plt.title('Plaquing log Titer vs 4h CFU Reduction (r = 0.57)')

# Show the plot
plt.show()

In [ ]:
screening_cnum = cnum_clinic[cnum_clinic["visit_raw"] == "Screening"]
screening_cnum = screening_cnum[screening_cnum['specimen_collection_time_description'] != "4 hr Post Dose"]
screening_cnum = screening_cnum.sort_values("subject_id")
six_prepause_isolates = ['c000675', 'c000678', 'c000681', 'c000686', 'c000687', 'c000692']
screening_cnum = screening_cnum[~screening_cnum["Name.."].isin(six_prepause_isolates)]

print(len(screening_cnum))


In [ ]:
want_these =  MiTT_patients[['Patient', 'Strain', 'Avg_logTiter_wZOC', '4h_NoPhage_vs_Phage', 'Patient_maxCFUdrop']]
want_these

In [ ]:
low_unique = unique_isos_summary[unique_isos_summary['Avg_logTiter_wZOC'] < 9]

In [ ]:
all_timepoints = ['Patient', 'Strain',"Patient_0h_logCFU", "Patient_4h_logCFU", "Patient_24h_logCFU", "Day3_logCFU", "Day4_logCFU", "Day5_logCFU", "Day10_logCFU", "Day34_logCFU", "Patient_maxCFUdrop", "Patient_maxCFUdrop_anyTime"]
MiTT_patients_cfu = MiTT_patients[all_timepoints]
MiTT_patients_cfu["dif_alltime_frombaseline"] = MiTT_patients_cfu["Patient_maxCFUdrop"] - MiTT_patients_cfu["Patient_maxCFUdrop_anyTime"]
MiTT_patients_cfu

In [ ]:
MiTT_patients_interest = MiTT_patients[['Patient', 'Strain', 'AUC_Ratio_LB', 'Avg_logTiter_wZOC', '4h_NoPhage_vs_Phage']]
MiTT_patients_interest

In [ ]:
BaselineIsos_Summary
#Scatter plots

# Extract the columns for the scatter plot
x_data = MiTT_patients['4h_NoPhage_vs_Phage']
y_data = MiTT_patients['Patient_maxCFUdrop_anyTime']

# Create a scatter plot
plt.scatter(x_data, y_data, color='blue', marker='o', label='Scatter Plot')

# Set labels and title
plt.xlabel('4h_NoPhage_vs_Phage')
plt.ylabel('Patient_maxCFUdrop_anyTime')
plt.title('4h_NoPhage_vs_Phage vs Patient_maxCFUdrop_anyTime')

# Show the plot
plt.show()

In [ ]:
BaselineIsos_PostRestart[["Strain", "Patient", "Phage", "4h_NoPhage_vs_Phage"]]

In [ ]:
BaselineIsos_PostRestart = BaselineIsos_Summary.dropna(subset=["24h_NoPhage_vs_Phage"])
BaselineIsos_PostRestart_strain_patient = BaselineIsos_PostRestart[["Strain", "Patient"]]
BaselineIsos_PostRestart_strain_patient["4h_CFUred_diff"] = BaselineIsos_PostRestart.groupby(["Strain", "Patient"])["4h_NoPhage_vs_Phage"].diff()
BaselineIsos_PostRestart_strain_patient["24h_CFUred_diff"] = BaselineIsos_PostRestart.groupby(["Strain", "Patient"])["24h_NoPhage_vs_Phage"].diff()
BaselineIsos_PostRestart_strain_patient = BaselineIsos_PostRestart_strain_patient.dropna()
baselineIsos_large_difference = BaselineIsos_PostRestart_strain_patient[BaselineIsos_PostRestart_strain_patient['24h_CFUred_diff'] < -1]
baselineIsos_large_difference = baselineIsos_large_difference.sort_values("Patient")
baselineIsos_large_difference

In [ ]:
BaselineIsos_618bactrim_FINALSTUFF = BaselineIsos_618bactrim[BaselineIsos_618bactrim.logCFU_24h < 3.5]
BaselineIsos_618bactrim_FINALSTUFF = BaselineIsos_618bactrim_FINALSTUFF[["Strain", "Patient", "logCFU_0h", "logCFU_4h", "logCFU_24h"]]
BaselineIsos_618bactrim_FINALSTUFF